In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from utils import session_config
os.environ['CUDA_VISIBLE_DEVICES'] = "1" # 사용 gpu 선택
session_config.setup_gpus(True, 0.5) # gpu 메모리 사용량 세팅

1 Physical GPUs, 1 Logical GPUs


In [3]:
%config Completer.use_jedi = False

In [4]:
import tensorflow as tf

# load your model already built at phase 1.
model = tf.keras.models.load_model("models/cifar_classifier.h5")

# load your dataset with tf.data
cifar10 = tf.keras.datasets.cifar10
(train_x, train_y), (test_x, test_y) = cifar10.load_data()

train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_ds = train_ds.shuffle(buffer_size = 20000, reshuffle_each_iteration=False)
val_ds = train_ds.take(10000)
train_ds = train_ds.skip(10000)

# Define customizing function for your model training

In [5]:
def custom_model(model):
    # your loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
    loss_tracker = tf.keras.metrics.Mean(name="loss")
    acc = tf.keras.metrics.SparseCategoricalAccuracy(name="acc")

    class CustomModel(tf.keras.Model):
        def train_step(self, data):
            x, y = data
            with tf.GradientTape() as tape:
                y_pred = self(x, training=True)  # Forward pass
                # Compute our own loss
                loss = loss_fn(y, y_pred)
                l2_loss = [tf.nn.l2_loss(t) for t in model.trainable_variables]
                l2_loss = self.l2_reg * tf.math.reduce_sum(l2_loss)
                total_loss = loss + l2_loss
            # Compute gradients
            trainable_vars = self.trainable_variables
            gradients = tape.gradient(total_loss, trainable_vars)

            # Update weights
            self.optimizer.apply_gradients(zip(gradients, trainable_vars))

            # Compute our own metrics
            loss_tracker.update_state(loss)
            acc.update_state(y, y_pred)
            return {"loss": loss_tracker.result(), "acc": acc.result()}
        def test_step(self, data):
            # Unpack the data
            x, y = data
            # Compute predictions
            y_pred = self(x, training=False)
            # Updates the metrics tracking the loss
            loss = loss_fn(y, y_pred)

            # Update the metrics.
            loss_tracker.update_state(loss)
            acc.update_state(y, y_pred)

            # Return a dict mapping metric names to current value.
            # Note that it will include the loss (tracked in self.metrics).
            return {m.name: m.result() for m in self.metrics}
        @property
        def metrics(self):
            # We list our `Metric` objects here so that `reset_states()` can be
            # called automatically at the start of each epoch
            # or at the start of `evaluate()`.
            # If you don't implement this property, you have to call
            # `reset_states()` yourself at the time of your choosing.
            return [loss_tracker, acc]
    return CustomModel(model.input, model.output)

# Hyper-parameter tuning

주요한 하이퍼파라미터는 다음과 같다. 

- optimizer = ["ADAM_W, "RMSPROP"]
- learning rate = [1E-3, 1E-4, 1E-5]
- learning rate scheduler = [constant, piecewise decay, linear decay, cosine decay restart]
- batch_size = [32, 64, 128]
- weight_decay = [1E-5, 5E-5, 1E-4]

리스트에 포함된 파라미터값에 의해 총 2x3x4x3x3 = 216가지 경우의 수에 대해 실험할 수 있다. 각 실험에 대해 Early stopping 지점까지 학습하고 validation 정확도를 측정한다. 우리는 주요한 하이퍼파라미터를 Grid search 방식으로 사전에 결정한다. 이런 방식은 하이퍼파라미터 튜닝에서 더 낮은 차원의 파라미터 공간에 대한 탐색을 수행할 수 있다. 이는 하이퍼파라미터 최적화에 대한 약간의 타협을 통해 효율적인 파라미터 튜닝을 가능하게 한다.

In [6]:
from utils.hparam_tune import Keras_Tuner

In [7]:
log_dir = "/home/files/AI_project_templates/logs/tuner_test5"
ckpt_dir = "/home/files/AI_project_templates/ckpts/tuner_test5"
initial_model_path = "models/cifar_classifier.h5"

tuner = Keras_Tuner(log_dir, ckpt_dir, initial_model_path, custom_model, train_ds, val_ds, metric="accuracy")

In [8]:
tuner.tuning()

--- Starting trial: run-0
{'init_lr': 1e-05, 'optimizer': 'ADAM_W', 'lr_scheduler': 'constant', 'batch_size': 32, 'weight_decay': 1e-05}
constant 1e-05
Epoch 1/300
1250/1250 [==============================] - 10s 5ms/step - loss: 1.6441 - acc: 0.4137 - val_loss: 1.3017 - val_acc: 0.5234

Epoch 00001: val_loss improved from inf to 1.30169, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-0/model-1.h5
Epoch 2/300
1250/1250 [==============================] - 6s 5ms/step - loss: 1.2061 - acc: 0.5698 - val_loss: 1.1679 - val_acc: 0.5881

Epoch 00002: val_loss improved from 1.30169 to 1.16792, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-0/model-2.h5
Epoch 3/300
1250/1250 [==============================] - 6s 5ms/step - loss: 1.0292 - acc: 0.6364 - val_loss: 1.0763 - val_acc: 0.6355

Epoch 00003: val_loss improved from 1.16792 to 1.07631, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-0/model-3.h5
Epoch 4/300
1250/1250 [

constant 1e-05
Epoch 1/300
1250/1250 [==============================] - 8s 5ms/step - loss: 1.6538 - acc: 0.4190 - val_loss: 1.3541 - val_acc: 0.5220

Epoch 00001: val_loss improved from inf to 1.35405, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-2/model-1.h5
Epoch 2/300
1250/1250 [==============================] - 6s 5ms/step - loss: 1.1949 - acc: 0.5793 - val_loss: 1.1320 - val_acc: 0.6016

Epoch 00002: val_loss improved from 1.35405 to 1.13199, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-2/model-2.h5
Epoch 3/300
1250/1250 [==============================] - 6s 5ms/step - loss: 1.0326 - acc: 0.6376 - val_loss: 1.0567 - val_acc: 0.6353

Epoch 00003: val_loss improved from 1.13199 to 1.05671, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-2/model-3.h5
Epoch 4/300
1250/1250 [==============================] - 6s 5ms/step - loss: 0.9220 - acc: 0.6784 - val_loss: 0.9797 - val_acc: 0.6616

Epoch 00004: val_loss imp

313/313 [==============================] - 3s 8ms/step - loss: 1.8346 - acc: 0.3814 - val_loss: 1.4858 - val_acc: 0.4487

Epoch 00001: val_loss improved from inf to 1.48577, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-6/model-1.h5
Epoch 2/300
313/313 [==============================] - 2s 5ms/step - loss: 1.3065 - acc: 0.5313 - val_loss: 1.2348 - val_acc: 0.5564

Epoch 00002: val_loss improved from 1.48577 to 1.23481, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-6/model-2.h5
Epoch 3/300
313/313 [==============================] - 2s 6ms/step - loss: 1.1273 - acc: 0.6008 - val_loss: 1.0913 - val_acc: 0.6087

Epoch 00003: val_loss improved from 1.23481 to 1.09132, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-6/model-3.h5
Epoch 4/300
313/313 [==============================] - 2s 6ms/step - loss: 0.9988 - acc: 0.6474 - val_loss: 1.0763 - val_acc: 0.6196

Epoch 00004: val_loss improved from 1.09132 to 1.07635, savi

313/313 [==============================] - 3s 7ms/step - loss: 1.8120 - acc: 0.3799 - val_loss: 1.4277 - val_acc: 0.4886

Epoch 00001: val_loss improved from inf to 1.42775, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-8/model-1.h5
Epoch 2/300
313/313 [==============================] - 2s 6ms/step - loss: 1.3024 - acc: 0.5357 - val_loss: 1.2348 - val_acc: 0.5576

Epoch 00002: val_loss improved from 1.42775 to 1.23475, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-8/model-2.h5
Epoch 3/300
313/313 [==============================] - 2s 6ms/step - loss: 1.1071 - acc: 0.6066 - val_loss: 1.1438 - val_acc: 0.5936

Epoch 00003: val_loss improved from 1.23475 to 1.14375, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-8/model-3.h5
Epoch 4/300
313/313 [==============================] - 2s 6ms/step - loss: 0.9739 - acc: 0.6581 - val_loss: 1.0967 - val_acc: 0.6140

Epoch 00004: val_loss improved from 1.14375 to 1.09668, savi

1250/1250 [==============================] - 6s 5ms/step - loss: 0.4413 - acc: 0.8469 - val_loss: 1.2042 - val_acc: 0.6699

Epoch 00014: val_loss did not improve from 0.98577
Epoch 15/300
1250/1250 [==============================] - 6s 5ms/step - loss: 0.4190 - acc: 0.8539 - val_loss: 1.2668 - val_acc: 0.6531

Epoch 00015: val_loss did not improve from 0.98577
Epoch 00015: early stopping
313/313 [==============================] - 1s 2ms/step - loss: 1.2668 - acc: 0.6531
--- Starting trial: run-12
{'init_lr': 1e-05, 'optimizer': 'ADAM_W', 'lr_scheduler': 'cosine_decay_restart', 'batch_size': 64, 'weight_decay': 1e-05}
cosine_decay_restart <tensorflow.python.keras.optimizer_v2.learning_rate_schedule.CosineDecayRestarts object at 0x7f12f6127eb8>
Epoch 1/300
625/625 [==============================] - 5s 6ms/step - loss: 1.7428 - acc: 0.3938 - val_loss: 1.3866 - val_acc: 0.4875

Epoch 00001: val_loss improved from inf to 1.38663, saving model to /home/files/AI_project_templates/ckpts/tuner_

...

313/313 [==============================] - 2s 6ms/step - loss: 0.7958 - acc: 0.7303 - val_loss: 1.1993 - val_acc: 0.5956

Epoch 00091: val_loss did not improve from 1.18776
Epoch 92/300
313/313 [==============================] - 2s 6ms/step - loss: 0.7909 - acc: 0.7322 - val_loss: 1.2010 - val_acc: 0.5892

Epoch 00092: val_loss did not improve from 1.18776
Epoch 93/300
313/313 [==============================] - 2s 6ms/step - loss: 0.7860 - acc: 0.7344 - val_loss: 1.2082 - val_acc: 0.5934

Epoch 00093: val_loss did not improve from 1.18776
Epoch 94/300
313/313 [==============================] - 2s 6ms/step - loss: 0.7819 - acc: 0.7376 - val_loss: 1.1934 - val_acc: 0.5936

Epoch 00094: val_loss did not improve from 1.18776
Epoch 95/300
313/313 [==============================] - 2s 7ms/step - loss: 0.7768 - acc: 0.7382 - val_loss: 1.1913 - val_acc: 0.5954

Epoch 00095: val_loss did not improve from 1.18776
Epoch 96/300
313/313 [==============================] - 2s 7ms/step - loss: 0.7714 


Epoch 00024: val_loss improved from 1.38691 to 1.37997, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-44/model-24.h5
Epoch 25/300
313/313 [==============================] - 2s 6ms/step - loss: 1.2608 - acc: 0.5567 - val_loss: 1.3698 - val_acc: 0.5190

Epoch 00025: val_loss improved from 1.37997 to 1.36975, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-44/model-25.h5
Epoch 26/300
313/313 [==============================] - 2s 6ms/step - loss: 1.2480 - acc: 0.5613 - val_loss: 1.3803 - val_acc: 0.5191

Epoch 00026: val_loss did not improve from 1.36975
Epoch 27/300
313/313 [==============================] - 2s 6ms/step - loss: 1.2367 - acc: 0.5669 - val_loss: 1.3654 - val_acc: 0.5204

Epoch 00027: val_loss improved from 1.36975 to 1.36535, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-44/model-27.h5
Epoch 28/300
313/313 [==============================] - 2s 6ms/step - loss: 1.2258 - acc: 0.5693 - val_loss: 1.3629 -

313/313 [==============================] - 2s 6ms/step - loss: 0.9706 - acc: 0.6651 - val_loss: 1.2339 - val_acc: 0.5683

Epoch 00059: val_loss improved from 1.23735 to 1.23390, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-44/model-59.h5
Epoch 60/300
313/313 [==============================] - 2s 6ms/step - loss: 0.9646 - acc: 0.6684 - val_loss: 1.2377 - val_acc: 0.5685

Epoch 00060: val_loss did not improve from 1.23390
Epoch 61/300
313/313 [==============================] - 2s 6ms/step - loss: 0.9593 - acc: 0.6701 - val_loss: 1.2358 - val_acc: 0.5700

Epoch 00061: val_loss did not improve from 1.23390
Epoch 62/300
313/313 [==============================] - 2s 6ms/step - loss: 0.9515 - acc: 0.6740 - val_loss: 1.2283 - val_acc: 0.5745

Epoch 00062: val_loss improved from 1.23390 to 1.22833, saving model to /home/files/AI_project_templates/ckpts/tuner_test5/run-44/model-62.h5
Epoch 63/300
313/313 [==============================] - 2s 6ms/step - loss: 0.9454 - ac

313/313 [==============================] - 2s 6ms/step - loss: 0.7605 - acc: 0.7422 - val_loss: 1.2027 - val_acc: 0.5981

Epoch 00098: val_loss did not improve from 1.18233
Epoch 99/300
313/313 [==============================] - 2s 6ms/step - loss: 0.7562 - acc: 0.7439 - val_loss: 1.1849 - val_acc: 0.6049

Epoch 00099: val_loss did not improve from 1.18233
Epoch 100/300
313/313 [==============================] - 2s 6ms/step - loss: 0.7516 - acc: 0.7473 - val_loss: 1.2079 - val_acc: 0.5960

Epoch 00100: val_loss did not improve from 1.18233
Epoch 101/300
313/313 [==============================] - 2s 6ms/step - loss: 0.7469 - acc: 0.7504 - val_loss: 1.2298 - val_acc: 0.5899

Epoch 00101: val_loss did not improve from 1.18233
Epoch 102/300
313/313 [==============================] - 2s 6ms/step - loss: 0.7433 - acc: 0.7501 - val_loss: 1.2239 - val_acc: 0.5953

Epoch 00102: val_loss did not improve from 1.18233
Epoch 103/300
313/313 [==============================] - 2s 6ms/step - loss: 0.7

TypeError: float() argument must be a string or a number, not 'CosineDecayRestarts'

In [ ]:

# model = tf.keras.models.load_model("ckpts/.../trained_model")
# test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y))
# test_ds = self.test_ds.batch(32).prefetch(tf.data.experimental.AUTOTUNE)
# model.evaluate(test_ds)